<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=f37abec2a7b21775df808993ecc7410968aceb0dd248515490795aabc0004d17
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=509463cbb6f08a1db5b2ffd24f818b87f07110f2dd1322ce8c8473dd1e28e666
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 222.82454562187195秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1195700,672.400024
1,ＩＮＰＥＸ,1605,鉱業,9330600,1767.000000
2,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,739000,45.000000
3,大林組,1802,建設業,1966800,1264.000000
4,清水建設,1803,建設業,3197400,927.000000
...,...,...,...,...,...
281,大阪瓦斯,9532,電気・ガス業,1289800,2166.500000
282,エイチ・アイ・エス,9603,サービス業,770200,1990.000000
283,エヌ・ティ・ティ・データ,9613,情報・通信業,1605300,1942.500000
284,ヤマダホールディングス,9831,小売業,2584600,428.700012


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1195700,672.400024
1,ＩＮＰＥＸ,1605,鉱業,9330600,1767.000000
2,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,739000,45.000000
3,大林組,1802,建設業,1966800,1264.000000
4,清水建設,1803,建設業,3197400,927.000000
...,...,...,...,...,...
206,北海道電力,9509,電気・ガス業,751100,565.099976
207,電源開発,9513,電気・ガス業,691000,2175.000000
208,東京瓦斯,9531,電気・ガス業,989300,3159.000000
209,エイチ・アイ・エス,9603,サービス業,770200,1990.000000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1195700,672.400024
1,ＩＮＰＥＸ,1605,鉱業,9330600,1767.000000
2,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,739000,45.000000
3,大林組,1802,建設業,1966800,1264.000000
4,清水建設,1803,建設業,3197400,927.000000
...,...,...,...,...,...
93,商船三井,9104,海運業,4546200,3713.000000
94,川崎汽船,9107,海運業,11805900,4188.000000
95,ＳＧホールディングス,9143,陸運業,1020600,2148.500000
96,ソフトバンク,9434,情報・通信業,5615600,1542.500000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,カルビー,2229,食料品,541600,2750.000000
1,明治ホールディングス,2269,食料品,827900,3352.000000
2,システナ,2317,情報・通信業,816700,288.000000
3,日本駐車場開発,2353,不動産業,2627000,219.000000
4,エムスリー,2413,サービス業,1603300,3089.000000
...,...,...,...,...,...
72,日本通信,9424,情報・通信業,3172800,234.000000
73,レノバ,9519,電気・ガス業,684300,1506.000000
74,大阪瓦斯,9532,電気・ガス業,1289800,2166.500000
75,ヤマダホールディングス,9831,小売業,2584600,428.700012


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,日本駐車場開発,2353,不動産業,2627000,219.000000
1,キリンホールディングス,2503,食料品,2290700,2076.000000
2,北の達人コーポレーション,2930,化学,1288200,260.000000
3,ＭｏｎｏｔａＲＯ,3064,小売業,1549300,1738.500000
4,ＺＯＺＯ,3092,小売業,2073400,2782.000000
5,ウエルシアホールディングス,3141,小売業,522800,2724.500000
6,ＦＯＯＤ ＆ ＬＩＦＥ ＣＯＭＰＡＮＩＥＳ,3563,小売業,904800,2631.500000
7,アイスタイル,3660,情報・通信業,5246600,489.000000
8,エニグモ,3665,情報・通信業,634600,364.000000
9,王子ホールディングス,3861,パルプ・紙,4353900,537.500000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,498400,2755.0
1,住石ホールディングス,1514,鉱業,748900,341.0
2,中小企業ホールディングス,1757,建設業,551800,40.0
3,アジアゲートホールディングス,1783,建設業,621200,53.0
4,土屋ホールディングス,1840,建設業,521600,243.0
...,...,...,...,...,...
85,ランド,8918,不動産業,27903800,8.0
86,明豊エンタープライズ,8927,不動産業,279400,244.0
87,ビジョナリーホールディングス,9263,小売業,270600,120.0
88,ビート・ホールディングス・リミテッド,9399,情報・通信業,641276,5.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,748900,341.0
1,中小企業ホールディングス,1757,建設業,551800,40.0
2,土屋ホールディングス,1840,建設業,521600,243.0
3,燦キャピタルマネージメント,2134,サービス業,4241400,24.0
4,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,541200,56.0
...,...,...,...,...,...
58,トレイダーズホールディングス,8704,証券、商品先物取引業,303500,641.0
59,エリアクエスト,8912,不動産業,7375800,117.0
60,ランド,8918,不動産業,27903800,8.0
61,明豊エンタープライズ,8927,不動産業,279400,244.0


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,土屋ホールディングス,1840,建設業,521600,243.000000
1,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,211500,608.000000
2,プラネット,2391,サービス業,459700,1341.000000
3,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,2649400,724.000000
4,出前館,2484,情報・通信業,1702100,442.000000
5,ベクター,2656,小売業,496300,349.000000
6,シー・ヴイ・エス・ベイエリア,2687,小売業,503800,809.000000
7,焼肉坂井ホールディングス,2694,小売業,261400,71.000000
8,ピクセルカンパニーズ,2743,卸売業,804900,55.000000
9,アップルインターナショナル,2788,卸売業,261800,358.000000


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,498400,2755.0
1,住石ホールディングス,1514,鉱業,748900,341.0
2,アジアゲートホールディングス,1783,建設業,621200,53.0
3,クルーズ,2138,情報・通信業,226100,867.0
4,クシム,2345,情報・通信業,697800,304.0
5,ゲンダイエージェンシー,2411,情報・通信業,256400,408.0
6,焼肉坂井ホールディングス,2694,小売業,261400,71.0
7,ｅｎｉｓｈ,3667,情報・通信業,700600,327.0
8,サイバーステップ,3810,情報・通信業,544100,388.0
9,リミックスポイント,3825,情報・通信業,624400,218.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,アジアゲートホールディングス,1783,建設業,621200,53.0
1,クルーズ,2138,情報・通信業,226100,867.0
2,クシム,2345,情報・通信業,697800,304.0
3,ゲンダイエージェンシー,2411,情報・通信業,256400,408.0
4,焼肉坂井ホールディングス,2694,小売業,261400,71.0
5,ｅｎｉｓｈ,3667,情報・通信業,700600,327.0
6,サイバーステップ,3810,情報・通信業,544100,388.0
7,リミックスポイント,3825,情報・通信業,624400,218.0
8,ハーモニック・ドライブ・システムズ,6324,機械,237600,4080.0
9,ＪＡＬＣＯホールディングス,6625,不動産業,227200,201.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,エムビーエス,1401,建設業,707400,516.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,272000,709.0
2,ジーエヌアイグループ,2160,医薬品,565000,1210.0
3,メディネット,2370,サービス業,805600,65.0
4,ウェッジホールディングス,2388,その他金融業,8151100,181.0
...,...,...,...,...,...
87,スマサポ,9342,サービス業,2091600,1858.0
88,ｉｓｐａｃｅ,9348,サービス業,402000,1350.0
89,リニューアブル・ジャパン,9522,電気・ガス業,4932700,1033.0
90,Ｍ＆Ａ総研ホールディングス,9552,サービス業,374600,3680.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,エムビーエス,1401,建設業,707400,516.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,272000,709.0
2,ジーエヌアイグループ,2160,医薬品,565000,1210.0
3,ウェッジホールディングス,2388,その他金融業,8151100,181.0
4,ジェイテック,2479,サービス業,610800,353.0
5,フルッタフルッタ,2586,食料品,277100,81.0
6,ベースフード,2936,食料品,3403100,517.0
7,海帆,3133,小売業,1970100,591.0
8,パルマ,3461,不動産業,1452100,381.0
9,ティーケーピー,3479,不動産業,273700,2724.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,エムビーエス,1401,建設業,707400,516.0
1,ウェッジホールディングス,2388,その他金融業,8151100,181.0
2,ジェイテック,2479,サービス業,610800,353.0
3,ベースフード,2936,食料品,3403100,517.0
4,ティーケーピー,3479,不動産業,273700,2724.0
5,霞ヶ関キャピタル,3498,不動産業,208400,4965.0
6,モブキャストホールディングス,3664,情報・通信業,3376900,91.0
7,アララ,4015,情報・通信業,496100,580.0
8,ティアンドエス,4055,情報・通信業,842600,3120.0
9,ＷＡＣＵＬ,4173,情報・通信業,367300,945.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,メディネット,2370,サービス業,805600,65.0
1,ＦＨＴホールディングス,3777,情報・通信業,622700,40.0
2,ディー・ディー・エス,3782,情報・通信業,1488300,11.0
3,Ａｉｍｉｎｇ,3911,情報・通信業,856800,441.0
4,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,202800,349.0
5,ココナラ,4176,情報・通信業,1135400,365.0
6,ＢＡＳＥ,4477,情報・通信業,1941700,245.0
7,フリー,4478,情報・通信業,268800,3095.0
8,ビザスク,4490,情報・通信業,205900,1249.0
9,アンジェス,4563,医薬品,240100,104.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,メディネット,2370,サービス業,805600,65.0
1,ディー・ディー・エス,3782,情報・通信業,1488300,11.0
2,ＢＡＳＥ,4477,情報・通信業,1941700,245.0
3,フリー,4478,情報・通信業,268800,3095.0
4,ビザスク,4490,情報・通信業,205900,1249.0
5,アンジェス,4563,医薬品,240100,104.0
6,キャンバス,4575,医薬品,278900,1129.0
7,サンバイオ,4592,医薬品,260400,592.0
8,ブライトパス・バイオ,4594,医薬品,452200,130.0
9,モダリス,4883,医薬品,585100,243.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,748900,341.000000
1,中小企業ホールディングス,1757,建設業,551800,40.000000
2,アジアゲートホールディングス,1783,建設業,621200,53.000000
3,土屋ホールディングス,1840,建設業,521600,243.000000
4,燦キャピタルマネージメント,2134,サービス業,4241400,24.000000
5,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,541200,56.000000
6,クシム,2345,情報・通信業,697800,304.000000
7,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,2649400,724.000000
8,出前館,2484,情報・通信業,1702100,442.000000
9,シー・ヴイ・エス・ベイエリア,2687,小売業,503800,809.000000


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,748900,341.0
1,中小企業ホールディングス,1757,建設業,551800,40.0
4,燦キャピタルマネージメント,2134,サービス業,4241400,24.0
5,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,541200,56.0
16,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,923300,2961.0
19,アルメタックス,5928,金属製品,510200,344.0
22,ヤマシナ,5955,金属製品,1082000,87.0
23,千代田化工建設,6366,建設業,632900,350.0
27,フェローテックホールディングス,6890,電気機器,1014200,3455.0
28,ヘリオス テクノ ホールディング,6927,電気機器,2193700,491.0


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,エムビーエス,1401,建設業,707400,516.0
1,ジーエヌアイグループ,2160,医薬品,565000,1210.0
2,メディネット,2370,サービス業,805600,65.0
3,ウェッジホールディングス,2388,その他金融業,8151100,181.0
4,ジェイテック,2479,サービス業,610800,353.0
5,ベースフード,2936,食料品,3403100,517.0
6,海帆,3133,小売業,1970100,591.0
7,パルマ,3461,不動産業,1452100,381.0
8,モブキャストホールディングス,3664,情報・通信業,3376900,91.0
9,ＦＨＴホールディングス,3777,情報・通信業,622700,40.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
1,ジーエヌアイグループ,2160,医薬品,565000,1210.0
6,海帆,3133,小売業,1970100,591.0
7,パルマ,3461,不動産業,1452100,381.0
9,ＦＨＴホールディングス,3777,情報・通信業,622700,40.0
11,Ａｉｍｉｎｇ,3911,情報・通信業,856800,441.0
13,プレイド,4165,情報・通信業,756900,827.0
14,ココナラ,4176,情報・通信業,1135400,365.0
15,エクサウィザーズ,4259,情報・通信業,1475300,457.0
18,オンコセラピー・サイエンス,4564,医薬品,8250500,32.0
21,ヘリオス,4593,医薬品,1142100,284.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')